In [4]:
import configparser
import datetime
import os
import tkinter as tk
from tkinter import messagebox, ttk

import mysql.connector
import pytz

# Importa IPython para obter o diretório do notebook
from IPython import get_ipython

class RefeicaoApp:
    """
    Aplicativo para registrar refeições em um banco de dados MySQL.
    """

    def __init__(self, root):
        """
        Inicializa a interface gráfica e configura a conexão com o banco de dados.

        Args:
            root (tk.Tk): A janela principal do aplicativo.
        """
        self.root = root
        self.root.title("Registro de Refeição")

        # Configurações do MySQL
        self.config = configparser.ConfigParser()

        # Determina o diretório do script e constrói o caminho para config.ini
        try:
            # Tenta obter o diretório se estiver rodando no IPython (Jupyter)
            script_dir = os.path.dirname(get_ipython().starting_dir)
        except AttributeError:
            # Se não estiver no IPython, usa o diretório atual de trabalho
            script_dir = os.getcwd()

        config_path = os.path.join(script_dir, "Credenciais", "config.ini")  # Garante o caminho correto
        self.config.read(config_path)

        self.db_config = self._load_db_config()
        if not self.db_config:
            root.destroy()
            return

        # Opções de tipo de refeição
        self.tipos_refeicao = ["café", "almoço", "janta", "lanche"]

        # Fuso horário de Brasília
        self.brasilia_tz = pytz.timezone("America/Sao_Paulo")

        # Componentes da Interface Gráfica
        self.inicio_label = ttk.Label(
            root, text="Hora de Início (HH:MM) - Formato 24h:"
        )
        self.inicio_hora_entry = ttk.Entry(root, width=5)
        self.inicio_minuto_entry = ttk.Entry(root, width=5)

        self.termino_label = ttk.Label(
            root, text="Hora de Término (HH:MM) - Formato 24h:"
        )
        self.termino_hora_entry = ttk.Entry(root, width=5)
        self.termino_minuto_entry = ttk.Entry(root, width=5)

        self.tipo_label = ttk.Label(root, text="Tipo de Refeição:")
        self.tipo_combobox = ttk.Combobox(
            root, values=self.tipos_refeicao, state="readonly"
        )

        self.salvar_button = ttk.Button(
            root, text="Salvar Registro de Refeição", command=self._salvar_refeicao
        )

        # Layout
        self.inicio_label.grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.inicio_hora_entry.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
        self.inicio_minuto_entry.grid(row=0, column=2, padx=5, pady=5, sticky="ew")

        self.termino_label.grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.termino_hora_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")
        self.termino_minuto_entry.grid(row=1, column=2, padx=5, pady=5, sticky="ew")

        self.tipo_label.grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.tipo_combobox.grid(row=2, column=1, columnspan=2, padx=5, pady=5, sticky="ew")

        self.salvar_button.grid(row=3, column=0, columnspan=3, padx=5, pady=5)

        root.columnconfigure(1, weight=1)
        root.columnconfigure(2, weight=1)

    def _load_db_config(self):
        """
        Carrega e valida a configuração do banco de dados do arquivo config.ini.

        Returns:
            dict: Um dicionário com as configurações do banco de dados.
                  Retorna None se houver um erro na configuração.
        """
        db_config = dict(self.config["database"])
        if "raise_on_warnings" in db_config:
            try:
                db_config["raise_on_warnings"] = self.config.getboolean(
                    "database", "raise_on_warnings"
                )
            except ValueError:
                messagebox.showerror(
                    "Erro",
                    "Valor inválido para raise_on_warnings no config.ini. "
                    "Use 'True' ou 'False'.",
                )
                return None
        return db_config

    def _validar_dados(self, inicio_hora_str, inicio_minuto_str,
                       termino_hora_str, termino_minuto_str, tipo_refeicao):
        """
        Valida os dados inseridos pelo usuário.

        Args:
            inicio_hora_str (str): A hora de início da refeição.
            inicio_minuto_str (str): O minuto de início da refeição.
            termino_hora_str (str): A hora de término da refeição.
            termino_minuto_str (str): O minuto de término da refeição.
            tipo_refeicao (str): O tipo de refeição.

        Returns:
            bool: True se os dados forem válidos, False caso contrário.
        """
        if not all(
            [inicio_hora_str, inicio_minuto_str, termino_hora_str,
             termino_minuto_str, tipo_refeicao]
        ):
            messagebox.showerror("Erro", "Por favor, preencha todos os campos.")
            return False

        try:
            inicio_hora = int(inicio_hora_str)
            inicio_minuto = int(inicio_minuto_str)
            termino_hora = int(termino_hora_str)
            termino_minuto = int(termino_minuto_str)

            if not (
                0 <= inicio_hora <= 23
                and 0 <= inicio_minuto <= 59
                and 0 <= termino_hora <= 23
                and 0 <= termino_minuto <= 59
            ):
                raise ValueError("Hora ou minuto inválido.")

        except ValueError as e:
            messagebox.showerror("Erro", f"Valor inválido: {e}")
            return False

        return True

    def _criar_objetos_datetime(self, inicio_hora_str, inicio_minuto_str,
                                 termino_hora_str, termino_minuto_str):
        """
        Cria objetos datetime para início e término da refeição.

        Args:
            inicio_hora_str (str): A hora de início da refeição.
            inicio_minuto_str (str): O minuto de início da refeição.
            termino_hora_str (str): A hora de término da refeição.
            termino_minuto_str (str): O minuto de término da refeição.

        Returns:
            tuple[datetime, datetime]: Uma tupla contendo os objetos
                                       datetime de início e término.
        """
        inicio_hora = int(inicio_hora_str)
        inicio_minuto = int(inicio_minuto_str)
        termino_hora = int(termino_hora_str)
        termino_minuto = int(termino_minuto_str)

        agora = datetime.datetime.now(self.brasilia_tz)
        hoje = agora.date()

        inicio = datetime.datetime.combine(hoje, datetime.time(inicio_hora, inicio_minuto))
        termino = datetime.datetime.combine(hoje, datetime.time(termino_hora, termino_minuto))

        inicio = self.brasilia_tz.localize(inicio)
        termino = self.brasilia_tz.localize(termino)

        return inicio, termino

    def _salvar_refeicao_no_banco(self, inicio, termino, tipo_refeicao):
        """
        Salva o registro de refeição no banco de dados.

        Args:
            inicio (datetime): O objeto datetime de início da refeição.
            termino (datetime): O objeto datetime de término da refeição.
            tipo_refeicao (str): O tipo de refeição.

        Returns:
            bool: True se o registro for salvo com sucesso, False caso contrário.
        """
        try:
            with mysql.connector.connect(**self.db_config) as connection:
                with connection.cursor() as cursor:
                    query = """
                        INSERT INTO habitos.refeicoes (inicio, termino, tipo)
                        VALUES (%s, %s, %s)
                    """
                    values = (
                        inicio.strftime("%Y-%m-%d %H:%M:%S"),
                        termino.strftime("%Y-%m-%d %H:%M:%S"),
                        tipo_refeicao,
                    )
                    cursor.execute(query, values)
                connection.commit()
            messagebox.showinfo("Sucesso", "Registro de refeição salvo com sucesso!")
            return True

        except mysql.connector.Error as err:
            messagebox.showerror("Erro", f"Erro ao salvar os dados: {err}")
            return False
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar o registro de refeição: {e}")
            return False

    def _salvar_refeicao(self):
        """
        Função principal para salvar o registro de refeição.
        """
        inicio_hora_str = self.inicio_hora_entry.get()
        inicio_minuto_str = self.inicio_minuto_entry.get()
        termino_hora_str = self.termino_hora_entry.get()
        termino_minuto_str = self.termino_minuto_entry.get()
        tipo_refeicao = self.tipo_combobox.get()

        if not self._validar_dados(
            inicio_hora_str, inicio_minuto_str, termino_hora_str,
            termino_minuto_str, tipo_refeicao
        ):
            return

        try:
            inicio, termino = self._criar_objetos_datetime(
                inicio_hora_str, inicio_minuto_str, termino_hora_str,
                termino_minuto_str
            )
        except ValueError as e:
            messagebox.showerror("Erro", f"Valor inválido: {e}")
            return

        if self._salvar_refeicao_no_banco(inicio, termino, tipo_refeicao):
            # Limpa os campos após o salvamento
            self.inicio_hora_entry.delete(0, tk.END)
            self.inicio_minuto_entry.delete(0, tk.END)
            self.termino_hora_entry.delete(0, tk.END)
            self.termino_minuto_entry.delete(0, tk.END)
            self.tipo_combobox.set("")


if __name__ == "__main__":
    root = tk.Tk()
    style = ttk.Style(root)
    style.theme_use("clam")
    app = RefeicaoApp(root)
    root.mainloop()